![darien](img\cover.jpg)

# Table of contents:
* [Data sources](#data_sources)
* [Total asylum seekers over the years](#total_asylum)
* [Migration crisis](#migration_crisis)
* [Country of origin](#country_of_origin)

#### You can find all the logic behind the notebook in the file: functions_and_backend.py

In [1]:
import functions_and_backend

# Data sources: <a class="anchor" id="data_sources">
<img src="img/sources.png"><br>


# From 1962 to 2023 more than <b>100,000,000<b> people have seek asylum

In [2]:
functions_and_backend.print_total_asylum_seekers()

108,224,695


# Countries that generate the most asylum seekers <a class="anchor" id="country_of_origin"></a>
Countries that generated less than 100,000 asylum seekers are not represented in the graph 

In [3]:
functions_and_backend.Country_of_origin()

## Asylum seekers over the years <a class="anchor" id="total_asylum"></a>
This graph shows how the asylum seeker population behave over the years, <br>
the highlighted zones represent an increment greater than 50%.<br>
Hovering over the graph lets you know which are the top 3 countries that contribute to that year migration.

In [4]:
functions_and_backend.New_asylum_seekers_graph()

# Closer look at the highlighted migration crisis <a class="anchor" id="migration_crisis"></a>
This is an interactive graph that allowes you to select the period that you would like to analyze

In [5]:
functions_and_backend.Migration_crisis_by_period_Dash()

# Countries that recived the most immigrants

In [6]:
functions_and_backend.Destination_countries_graph()

# Biggest porcentual population displacement in one year

In [7]:
functions_and_backend.Biggest_displacement_percentage_graph()

# Specific country migration graphs

In [8]:
functions_and_backend.Specific_country_information_dash()